In [12]:
using WebIO

In [2]:
node(:div, "Hello, World")

(div
  "Hello, World")

# Composing content

Let's say you want to display the following HTML:

```html
<ul class="my-list">
    <li>get milk</li>
    <li>make a pie</li>
</ul>
```

You can create a nested Node object:

In [ ]:
node(:ul,
    node(:li, "get milk"),
    node(:li, "make a pie"), attributes=Dict(:class => "my-list"))

# Loading JavaScript dependencies

You can load dependencies by creating a Scope object and passing in `imports` argument.

In [19]:
w = Scope(imports=["//people.csail.mit.edu/rdeits/webio_tests/trivial_import.js"])

inbox = Observable{Any}(w, "inbox", nothing)
on(inbox) do x
    @info "Inbox got \"$x\"."
end

onimport(w, js"""
function (dependency) {
    console.log(dependency);
    console.log(_webIOScope);
    _webIOScope.setObservableValue("inbox", dependency.x);
}
""")

n = w(dom"div#container"())
display(n)

# There's no real way (that I can see) to do this test synchronously since we
# need to be idle for IJulia to handle updating the observable but we also
# can't throw from within an async block and have Jupyter pick it up.
@async begin
    sleep(3)
    if inbox[] == "ok"
        @info "Passed!"
    else
        @warn "Failed!"
    end
end

Scope("scope-391c2f6d-07c7-404d-928f-c9681106d8c5", Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[], Dict{Symbol,Any}(:id=>"container"), 0), Dict{String,Tuple{Observables.AbstractObservable,Union{Nothing, Bool}}}("inbox"=>(Observable{Any}("ob_12", nothing, Any[##55#56()]), nothing)), Set(String[]), nothing, ["//people.csail.mit.edu/rdeits/webio_tests/trivial_import.js"], Dict{Any,Any}("_promises"=>Dict{Any,Any}("importsLoaded"=>Any[JSString("function (dependency) {\n    console.log(dependency);\n    console.log(_webIOScope);\n    _webIOScope.setObservableValue(\"inbox\", dependency.x);\n}\n")])), WebIO.ConnectionPool(Channel{Any}(sz_max:9223372036854775807,sz_curr:0), Set(AbstractConnection[]), Channel{AbstractConnection}(sz_max:32,sz_curr:0)))

Task (runnable) @0x00007f91a254bc70

┌ Info: Inbox got "ok".
└ @ Main In[19]:5
┌ Info: Passed!
└ @ Main In[19]:22


# Sending values from JavaScript to Julia

Below is a scope which communicates with Julia. The following scope contains a button which sends a random number, generated in JavaScript, to Julia. We will print this number on the Julia side.

In [22]:
function random_print_button()
    w = Scope()

    obs = Observable(w, "rand-value", 0.0)

    on(obs) do x
        println("JS sent $x")
    end

    w(
      dom"button"(
        "generate random",
        events=Dict("click"=>js"""function() { _webIOScope.setObservableValue("rand-value", Math.random()); }"""),
      ),
    )
end

random_print_button (generic function with 1 method)

In [23]:
random_print_button()

Scope("scope-06b9de5b-4f7c-4d05-a82d-9228e697f121", Node{WebIO.DOM}(WebIO.DOM(:html, :button), Any["generate random"], Dict{Symbol,Any}(:events=>Dict("click"=>JSString("function() { _webIOScope.setObservableValue(\"rand-value\", Math.random()); }"))), 1), Dict{String,Tuple{Observables.AbstractObservable,Union{Nothing, Bool}}}("rand-value"=>(Observable{Float64}("ob_13", 0.0, Any[##61#63()]), nothing)), Set(String[]), nothing, Any[], Dict{Any,Any}(), WebIO.ConnectionPool(Channel{Any}(sz_max:9223372036854775807,sz_curr:0), Set(AbstractConnection[]), Channel{AbstractConnection}(sz_max:32,sz_curr:0)))

JS sent 0.13925483172939013
JS sent 0.024235441221493348
JS sent 0.4880460014322092
JS sent 0.6848227695164495


## iframe encapsulation

We can also encapsulate a scope inside an `<iframe>`, which isolates it from the containing page's styling and layout: 

In [24]:
iframe(random_print_button())

(WebIO.IFrame("<div\n    class=\"webio-mountpoint\"\n    data-webio-mountpoint=\"12573687166207745325\"\n>\n    <script>\n    if (window.require && require.defined && require.defined(\"nbextensions/webio/main\")) {\n        console.log(\"Jupyter WebIO extension detected, not mounting.\");\n    } else if (window.WebIO) {\n        WebIO.mount(\n            document.querySelector('[data-webio-mountpoint=\"12573687166207745325\"]'),\n            {\"props\":{},\"nodeType\":\"Scope\",\"type\":\"node\",\"instanceArgs\":{\"imports\":{\"data\":[],\"type\":\"async_block\"},\"id\":\"scope-215e5f1c-a7fb-4dcc-9737-435f6e218b33\",\"handlers\":{},\"systemjs_options\":null,\"observables\":{\"rand-value\":{\"sync\":true,\"id\":\"ob_14\",\"value\":0.0}}},\"children\":[{\"props\":{\"events\":{\"click\":\"function() { _webIOScope.setObservableValue(\\\"rand-value\\\", Math.random()); }\"}},\"nodeType\":\"DOM\",\"type\":\"node\",\"instanceArgs\":{\"namespace\":\"html\",\"tag\":\"button\"},\"children\":[\"generate random\"]}]},\n        );\n    } else {\n        document\n            .querySelector('[data-webio-mountpoint=\"12573687166207745325\"]')\n            .innerHTML = '<strong>WebIO not detected.</strong>';\n    }\n    </script>\n</div>\n"):)

JS sent 0.5822867623495988
JS sent 0.2613272185876856
JS sent 0.3942177563282824


# Sending values from Julia to JavaScript

Here's a clock where the time is formatted and updated every second from Julia. We use the onjs handler and mutate the #clock DOM element to acheive this.

In [25]:
using Dates
w = Scope()
obs = Observable(w, "clock-value", "")

timestr() = Dates.format(now(), "HH:MM:SS")

# update timestamp every second
@async while true
    sleep(1)
    obs[] = timestr()
end

onjs(obs, js"""
function (value) {
    const clock = this.dom.querySelector(".clock");
    clock.textContent = value;
}
""")

w(
  dom"div.clock"(
    timestr(),
  ),
)

Scope("scope-50c58b2e-82ab-4705-b578-7b0cf8086797", Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any["10:46:10"], Dict{Symbol,Any}(:className=>"clock"), 1), Dict{String,Tuple{Observables.AbstractObservable,Union{Nothing, Bool}}}("clock-value"=>(Observable{String}("ob_15", "", Any[SyncCallback(Scope(#= circular reference @-6 =#), SyncCallback(Scope(#= circular reference @-7 =#), ##36#38{Scope,String}(Scope(#= circular reference @-8 =#), "clock-value")))]), nothing)), Set(String[]), nothing, Any[], Dict{Any,Any}("clock-value"=>Any[JSString("function (value) {\n    const clock = this.dom.querySelector(\".clock\");\n    clock.textContent = value;\n}\n")]), WebIO.ConnectionPool(Channel{Any}(sz_max:9223372036854775807,sz_curr:0), Set(AbstractConnection[]), Channel{AbstractConnection}(sz_max:32,sz_curr:0)))

For an even easier way to send values from Julia to JavaScript, we can simply rely on the fact that WebIO knows how to render `Observable`s directly to HTML. In this case WebIO will automatically construct a `Scope` and insert the relevant JavaScript to update the rendered content whenever the `Observable` changes value:

In [26]:
clock_obs = Observable(timestr())
@async while true
    sleep(1)
    clock_obs[] = timestr()
end
clock_obs

Observable{String}("ob_16", "10:46:36", Any[])